In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split # pip install scikit-learn==1.5.0
import keras
from sklearn.metrics import accuracy_score, confusion_matrix

In [4]:
"""Atributos previsores (x)"""
x = pd.read_csv('/home/vitor-lima/Documents/jupyter/deep_learning_com_python_de_a_a_z/breast_cancer_wisconsin_diagnostic/x.csv')

In [5]:
"""Função para preencher as colunas das classes (y) com base no diagnóstico"""
def binary(row):
    if row['diagnosis'] == 'M':
        return 1
    return 0

In [6]:
"""Criando o arquivo binário"""
y = x.apply(binary, axis=1)
y = pd.DataFrame(y, columns=['binary'])
y.to_csv('/home/vitor-lima/Documents/jupyter/deep_learning_com_python_de_a_a_z/breast_cancer_wisconsin_diagnostic/y.csv', index=False)
x.drop(['id', 'diagnosis'], axis='columns', inplace=True)

In [7]:
x_training, x_test, y_training, y_test = train_test_split(x, y, test_size=0.25) # 25% para testar e 75% para treinar
x_training.to_csv('/home/vitor-lima/Documents/jupyter/deep_learning_com_python_de_a_a_z/breast_cancer_wisconsin_diagnostic/x_training.csv')
y_training.to_csv('/home/vitor-lima/Documents/jupyter/deep_learning_com_python_de_a_a_z/breast_cancer_wisconsin_diagnostic/y_training.csv')
x_test.to_csv('/home/vitor-lima/Documents/jupyter/deep_learning_com_python_de_a_a_z/breast_cancer_wisconsin_diagnostic/x_test.csv')
y_test.to_csv('/home/vitor-lima/Documents/jupyter/deep_learning_com_python_de_a_a_z/breast_cancer_wisconsin_diagnostic/y_test.csv')

In [8]:
"""Criando a variável que representa a rede neural"""
neural_net = keras.Sequential([
    keras.layers.InputLayer(shape=(10,)), # shape deve ser igual ao número de atributos previsores
    keras.layers.Dense(units=6, activation='relu', kernel_initializer='random_uniform'), # Rede neural densa. Todos os neurônios de uma camada estão conectados aos neurônios da próxima camada. Para definir o parâmetro units, fazemos a média entre o número de neurônios de entrada e de saída, sempre arredondando para cima
    keras.layers.Dense(units=6, activation='relu', kernel_initializer='random_uniform'), # Podemos adicionar mais de uma camada oculta. Isso é recomendado para problemas mais complexos
    keras.layers.Dense(units=1, activation='sigmoid') # Camada de saída
])

2025-03-26 13:41:43.891821: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [9]:
neural_net.summary()
## Com uma camada oculta
# (66) Param = 6 [neurônios da camada oculta] * 10 [número de parâmetros] + 6 [bias]
# (7) Param = 6 [neurônios da camada oculta] + 1 [bias]
# (73) Total params = 66 + 7. Representam os pesos que serão ajustados

## Com duas camadas ocultas
# (66) Param = 6 [neurônios da primeira camada oculta] * 10 [número de parâmetros] + 6 [bias]
# (42) Param = 6 [neurônios da primeira camada oculta] * 6 [neurônios da segunda camada oculta] + 6 [bias]
# (7) Param = 6 [neurônios da segunda camada oculta] + 1 [bias]
# (115) Total params = 66 + 42 + 7. Representam os pesos que serão ajustados

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 6)              │            66 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │            42 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │             7 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 115 (460.00 B)

 Trainable params: 115 (460.00 B)

 Non-trainable params: 0 (0.00 B)

In [10]:
w0 = neural_net.layers[0].get_weights() # Valores dos pesos entre a entrada e a primeira camada oculta + bias
w0 # = 2 arrays. O primeiro apresenta os valores dos pesos das ligações entre os neurônios da camada de entrada e a primeira camada oculta. O segundo, representa os pesos das ligações entre unidade de bias e a primeira camada oculta

[array([[-6.7923218e-04,  1.5049610e-02, -3.8442612e-03, -2.8914904e-02,
         -4.8916567e-02,  1.5307665e-03],
        [ 4.3817941e-02, -1.6341187e-02, -4.6752762e-02, -3.7253983e-03,
          4.6815645e-02, -4.2637732e-02],
        [ 2.1761145e-02, -3.3618346e-02, -2.8968966e-02, -3.6531795e-02,
         -2.8581297e-02, -2.4444349e-03],
        [ 1.2423288e-02,  4.6999339e-02,  1.8426999e-03,  4.4144992e-02,
          4.8034672e-02,  7.2650984e-04],
        [ 4.5759384e-02, -3.6120642e-02,  8.5406415e-03,  3.5323944e-02,
          1.9521046e-02, -1.3030879e-03],
        [-8.7278970e-03,  3.3827636e-02, -2.4422122e-02, -4.7796633e-02,
          4.8280504e-02,  4.9289707e-02],
        [-2.6153756e-02, -2.9510248e-02,  2.2103675e-03,  1.7217923e-02,
          1.6922466e-03, -3.1341694e-02],
        [-1.7036580e-02, -3.8075756e-02, -2.8999878e-02, -2.8339518e-02,
          2.1441344e-02,  3.2178249e-02],
        [ 1.8405806e-02,  1.2318045e-04, -2.2926703e-03, -4.0072631e-02,
       

In [11]:
w1 = neural_net.layers[1].get_weights() # Valores dos pesos entre a primeira e a segunda camada oculta + bias
w1 # = 2 arrays. O primeiro apresenta os valores dos pesos das ligações entre os neurônios da primeira e da segunda camada oculta. O segundo, representa os pesos das ligações entre unidade de bias e a segunda camada oculta

[array([[ 0.04195334, -0.03529788,  0.01488503, -0.04265226, -0.02930948,
         -0.02018313],
        [ 0.03373046,  0.03604451, -0.04867604,  0.04654991, -0.04753604,
         -0.00813355],
        [-0.01014822,  0.0137903 ,  0.03867828,  0.04359471,  0.03125766,
         -0.00818855],
        [ 0.04050593, -0.02274437,  0.01950828,  0.01837832, -0.02121673,
          0.0448862 ],
        [ 0.0333542 , -0.03258743, -0.02746263, -0.04977874, -0.04752619,
         -0.00988712],
        [ 0.01912122,  0.03808368,  0.00917013,  0.03763303,  0.00889998,
          0.02862147]], dtype=float32),
 array([0., 0., 0., 0., 0., 0.], dtype=float32)]

In [12]:
w2 = neural_net.layers[2].get_weights() # Valores dos pesos entre a primeira e a segunda camada oculta + bias
w2 # = 2 arrays. O primeiro apresenta os valores dos pesos das ligações entre os neurônios da segunda camada oculta e os neurônio da camada desaída. O segundo, representa o peso da ligação entre a unidade de bias e o neurônio da camada de saída

[array([[-0.34194434],
        [ 0.9063628 ],
        [ 0.12486959],
        [-0.49472648],
        [ 0.12654161],
        [ 0.5827278 ]], dtype=float32),
 array([0.], dtype=float32)]

In [13]:
optmizer = keras.optimizers.Adam(learning_rate=0.001, clipvalue=0.5)
# learning_rate = Quanto maior a taxa, mais rápido a rede é treinada, porém, ela talvez não consiga encontrar o mínimo global
# clipvalue = Limite para o valor dos pesos (os pesos não serão maiores que esse valor). Útil em casos nos quais os resultados não estão tão bons

In [14]:
"""Definindo a maneira de ajuste dos pesos, função que calcula o erro, e as métricas retornadas"""
neural_net.compile(optimizer=optmizer, loss='binary_crossentropy', metrics=['binary_accuracy'])
# optimizer = Algoritmo utilizado para o ajuste dos pesos. Adam é uma melhoria do Sthocastic Gradient Descent
# loss = Função que calcula o erro. BinaryCrossentropy para duas classes. CategoricalCrossentropy para mais de duas classes. MeanAbsolute e MeanSquare são utilizadas em regressões (previsões de números). As funções baseadas em Crossentropy utilizam cálculos de logaritmos, sendo mais otmizadas e tendo um desempenho melhor
# metrics = Lista com as métricas que serão retornadas. BinaryAccuracy para problemas de classificação binária

In [15]:
"""Treinando o modelo"""
neural_net.fit(x_training, y_training, batch_size=10, epochs=100)
# batch_size = Quantidade de dados que vai ser enviada por vez para a rede neural. Os pesos são atualizados após o processamento de um batch que, nesse caso, é composto por 10 registros
# epochs = Quantidade de vezes que o processo será executado. Em cada uma das vezes, a rede neural irá processar todos os registros

Epoch 1/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - binary_accuracy: 0.5451 - loss: 0.7245
Epoch 2/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.7553 - loss: 0.6545
Epoch 3/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.6596 - loss: 0.6094
Epoch 4/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.7697 - loss: 0.5813
Epoch 5/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.8172 - loss: 0.5322
Epoch 6/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.7708 - loss: 0.5135
Epoch 7/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.8231 - loss: 0.4680
Epoch 8/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.8575 - loss: 0.4291
Epoch 9/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.8173 - loss: 0.4692
Epoch 10/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.8044 - loss: 0.4238
Epoch 11/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.8476 -

In [16]:
"""Previsões e avaliação da base de dados de teste"""
predictions = neural_net.predict(x_test)
predictions = predictions > 0.5 # Converte os valores retornados pela função sigmoide (valores estes que estão entre 0 e 1) para True ou False. Quanto maior o valor, maior a confiança que esperamos do algoritmo
predictions_df = pd.DataFrame(predictions, columns=['predictions'])
predictions_df.to_csv('/home/vitor-lima/Documents/jupyter/deep_learning_com_python_de_a_a_z/breast_cancer_wisconsin_diagnostic/predictions.csv')

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


In [17]:
"""Avaliando a acurácia do modelo"""
accuracy_score(y_test, predictions)

0.9020979020979021

In [18]:
"""Visualizando a matriz de erro"""
confusion_matrix(y_test, predictions)
# Primeira linha e primeira coluna = Classe 0
# Sgunda linha e segunda coluna = Classe 1
# [1, 1] tumores benignos foram classificados corretamente, e [1, 2] não
# [2, 2] tumores malignos foram classificados corretamente, e [2, 1] não
# [1, 1] + [2, 2] = Total de acertos
# [1, 2] + [2,1 ] = Total de erros

array([[91,  3],
       [11, 38]])

In [19]:
neural_net.evaluate(x_test, y_test) # Outra forma de avaliar o valor do erro e a acurácia

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - binary_accuracy: 0.9157 - loss: 0.2149  


[0.22492770850658417, 0.9020978808403015]